In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
import numpy as np
import keras
import cv2
import matplotlib.pyplot as plt
import sys,gc
import pickle
import pandas as pd

C:\Users\sajan\AppData\Local\Programs\Python\Python37\Lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\sajan\AppData\Local\Programs\Python\Python37\Lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\sajan\AppData\Local\Programs\Python\Python37\Lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\sajan\AppD

In [2]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import f1_score, accuracy_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split

In [3]:
train = pd.read_csv('train.csv')
train_labels = train.pop('air_pollution_index')
train.head()

,date_time,is_holiday,humidity,wind_speed,wind_direction,visibility_in_miles,dew_point,temperature,rain_p_h,snow_p_h,clouds_all,weather_type,traffic_volume
0,2012-10-02 09:00:00,None,89,2,329,1,1,288.28,0.0,0.0,40,Clouds,5545
1,2012-10-02 10:00:00,None,67,3,330,1,1,289.36,0.0,0.0,75,Clouds,4516
2,2012-10-02 11:00:00,None,66,3,329,2,2,289.58,0.0,0.0,90,Clouds,4767
3,2012-10-02 12:00:00,None,66,3,329,5,5,290.13,0.0,0.0,90,Clouds,5026
4,2012-10-02 13:00:00,None,65,3,329,7,7,291.14,0.0,0.0,75,Clouds,4918


In [4]:
test = pd.read_csv('test.csv')
test.head()

,date_time,is_holiday,humidity,wind_speed,wind_direction,visibility_in_miles,dew_point,temperature,rain_p_h,snow_p_h,clouds_all,weather_type,traffic_volume
0,2017-05-18 00:00:00,None,63,1,27,4,4,285.15,0.0,0,90,Rain,646
1,2017-05-18 00:00:00,None,63,1,27,4,4,285.15,0.0,0,90,Mist,646
2,2017-05-18 00:00:00,None,56,1,0,1,1,285.15,0.0,0,90,Drizzle,646
3,2017-05-18 01:00:00,None,56,1,351,2,2,284.79,0.0,0,90,Rain,353
4,2017-05-18 01:00:00,None,56,1,351,1,1,284.79,0.0,0,90,Mist,353


In [6]:
train_test = pd.concat([train,test])
train_test.shape

(48204, 13)

In [7]:
print(pd.unique(train_test.is_holiday))
print(train_test.is_holiday.value_counts())
train_test.is_holiday = (train_test.is_holiday != 'None').astype(np.int8)
print(train_test.is_holiday.value_counts())

['None' 'Columbus Day' 'Veterans Day' 'Thanksgiving Day' 'Christmas Day'
 'New Years Day' 'Washingtons Birthday' 'Memorial Day' 'Independence Day'
 'State Fair' 'Labor Day' 'Martin Luther King Jr Day']
None                         48143
Labor Day                        7
Christmas Day                    6
New Years Day                    6
Thanksgiving Day                 6
Martin Luther King Jr Day        6
Independence Day                 5
Washingtons Birthday             5
Columbus Day                     5
State Fair                       5
Veterans Day                     5
Memorial Day                     5
Name: is_holiday, dtype: int64
0    48143
1       61
Name: is_holiday, dtype: int64


In [8]:
train_test = pd.concat([train_test, pd.get_dummies(train_test.weather_type)], axis = 1 )
train_test.drop(['weather_type'], axis = 1, inplace=True)

In [9]:
train_test.date_time = pd.to_datetime(train_test.date_time)

In [25]:
train_test['day'] = train_test.date_time.dt.day
train_test['month'] = train_test.date_time.dt.month
train_test['year'] = train_test.date_time.dt.year
train_test['hour'] = train_test.date_time.dt.hour
# train_test['minute'] = train_test.date_time.dt.minute
# train_test['second'] = train_test.date_time.dt.second


In [37]:
train_test.drop('date_time', axis=1, inplace=True)

In [38]:
train_test.head()

,is_holiday,humidity,wind_speed,wind_direction,visibility_in_miles,dew_point,temperature,rain_p_h,snow_p_h,clouds_all,...,Mist,Rain,Smoke,Snow,Squall,Thunderstorm,day,month,year,hour
0,0,89,2,329,1,1,288.28,0.0,0.0,40,...,0,0,0,0,0,0,2,10,2012,9
1,0,67,3,330,1,1,289.36,0.0,0.0,75,...,0,0,0,0,0,0,2,10,2012,10
2,0,66,3,329,2,2,289.58,0.0,0.0,90,...,0,0,0,0,0,0,2,10,2012,11
3,0,66,3,329,5,5,290.13,0.0,0.0,90,...,0,0,0,0,0,0,2,10,2012,12
4,0,65,3,329,7,7,291.14,0.0,0.0,75,...,0,0,0,0,0,0,2,10,2012,13


In [41]:
stdscaler = StandardScaler()
train_test = stdscaler.fit_transform(train_test)

In [75]:
reg = RandomForestRegressor(random_state=5,verbose=1,n_jobs=8, n_estimators = 4000, warm_start=True, max_depth=8)
reg.fit(train_test[:train.shape[0]], train_labels)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    1.6s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    9.4s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:   20.5s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:   37.0s
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:   56.3s
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:  1.3min
[Parallel(n_jobs=8)]: Done 2434 tasks      | elapsed:  1.8min
[Parallel(n_jobs=8)]: Done 3000 out of 3000 | elapsed:  2.1min finished


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=8, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=3000, n_jobs=8, oob_score=False,
                      random_state=5, verbose=1, warm_start=True)

In [76]:
preds = reg.predict(train_test[train.shape[0]:])

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:    0.8s
[Parallel(n_jobs=8)]: Done 2434 tasks      | elapsed:    1.1s
[Parallel(n_jobs=8)]: Done 3000 out of 3000 | elapsed:    1.3s finished


In [ ]:
gbt = GradientBoostingRegressor(random_state=5,verbose=1,max_depth=8, n_estimators = 3000,  criterion='mse',warm_start=True,tol=0.00001,learning_rate=0.05)
gbt.fit(train_test[:train.shape[0]], train_labels)

In [94]:
gbt_preds = gbt.predict(train_test[train.shape[0]:])

In [95]:
sub = pd.DataFrame()

In [96]:
sub['date_time'] = test.date_time

In [101]:
sub['air_pollution_index'] = np.mean([preds,gbt_preds]).astype('int')#np.array(gbt_preds).astype('int')#

In [102]:
sub.to_csv('sub.csv',index=False)

In [82]:
# GradientBoostingRegressor()

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)